In [11]:
import tensorflow as tf
import keras
import numpy as np

# Training dataset
training_set= keras.utils.image_dataset_from_directory(
    'C:/Users/parth/Desktop/Desktop/Concordia/6180/Dataset/malimg_dataset/train/',
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
)

valid_data= keras.utils.image_dataset_from_directory(
    'C:/Users/parth/Desktop/Desktop/Concordia/6180/Dataset/malimg_dataset/val/',
    labels="inferred",
    label_mode="categorical",
    color_mode="grayscale",
)


def identitiy_block(data, filter):
    # Copy input tensor to a variable
    data_skip = data
   
    # Build first Layer
    data = tf.keras.layers.Conv2D(filter, (1,1), padding= 'same')(data)
    data = tf.keras.layers.BatchNormalization(axis=3)(data)
    data = tf.keras.layers.Activation('relu')(data)
    # Build second Layer
    data = tf.keras.layers.Conv2D(filter, (3,3), padding= 'same')(data)
    data = tf.keras.layers.BatchNormalization(axis=3)(data)
    data = tf.keras.layers.Activation('relu')(data)
    # print(data)
    # Build third layer
    data = tf.keras.layers.Conv2D(filter*4, (1,1), padding= 'same')(data)
    data = tf.keras.layers.BatchNormalization(axis=3)(data)
    # data = tf.keras.layers.Conv2D(filter, (1,1), padding= 'same')(data)

    # 64,64,256
    
    # Adding Resedue
    data = tf.keras.layers.add([data, data_skip])
    data = tf.keras.layers.Activation('relu')(data)
    return data
def Convolutional_block(data, filter, strides=(2,2)):
    # Copy input tensor to a variable
    data_skip = data
    # Build first Layer
    data = tf.keras.layers.Conv2D(filter, (1,1), padding= 'same', strides= strides)(data)
    data = tf.keras.layers.BatchNormalization()(data)
    data = tf.keras.layers.Activation('relu')(data)
    # Build second Layer
    data = tf.keras.layers.Conv2D(filter, (3,3), padding= 'same')(data)
    data = tf.keras.layers.BatchNormalization()(data)
    data = tf.keras.layers.Activation('relu')(data)
    # Buold third layer
    data = tf.keras.layers.Conv2D(filter*4, (1,1), padding= 'same')(data)
    data = tf.keras.layers.BatchNormalization()(data)
    # Adding residue
    data_skip = tf.keras.layers.Conv2D(filter*4, (1,1), strides=strides)(data_skip)
    data = tf.keras.layers.Add()([data, data_skip])     
    data = tf.keras.layers.Activation('relu')(data)
    return data
def ResNet50(datashape, classes):
    data_input = tf.keras.Input(shape=datashape)

    data = tf.keras.layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(data_input)
    # Initial Conv layer
    data = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(data_input)
    data = tf.keras.layers.BatchNormalization()(data)
    data = tf.keras.layers.Activation('relu')(data)
    data = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(data)
    # Starting Conv and Identity layers
    data = Convolutional_block(data, 64, strides=(1,1))
    data = identitiy_block(data, 64)
    data = identitiy_block(data, 64)
    # Starting second major layer
    data = Convolutional_block(data, 128)
    data = identitiy_block(data, 128)
    data = identitiy_block(data, 128)
    data = identitiy_block(data, 128)
    # Starting third major layer
    data = Convolutional_block(data, 246)
    data = identitiy_block(data, 246)
    data = identitiy_block(data, 246)
    data = identitiy_block(data, 246)
    data = identitiy_block(data, 246)
    data = identitiy_block(data, 246)
    # Starting four major layer
    data = Convolutional_block(data, 512)
    data = identitiy_block(data, 512)
    data = identitiy_block(data, 512)


   



    # Step 4 End Dense Network
    data = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(data)
    data = tf.keras.layers.Flatten()(data)
    data = tf.keras.layers.Dense(512, activation = 'relu')(data)
    data = tf.keras.layers.Dense(classes, activation = 'softmax')(data)
    model = tf.keras.models.Model(inputs = data_input, outputs = data, name = "ResNet50")
    return model
model = ResNet50((256,256,1), 25)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.fit(training_set, validation_data=valid_data, epochs = 5)




Found 7459 files belonging to 25 classes.
Found 923 files belonging to 25 classes.
Epoch 1/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 561s 2s/step - accuracy: 0.4883 - loss: 8.8581 - val_accuracy: 0.3185 - val_loss: 9150.9902
Epoch 2/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - accuracy: 0.8792 - loss: 0.4621 - val_accuracy: 0.9480 - val_loss: 0.1991
Epoch 3/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 553s 2s/step - accuracy: 0.9302 - loss: 0.2284 - val_accuracy: 0.9458 - val_loss: 0.1540
Epoch 4/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 498s 2s/step - accuracy: 0.9502 - loss: 0.1493 - val_accuracy: 0.6750 - val_loss: 2.4952
Epoch 5/5
234/234 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - accuracy: 0.8697 - loss: 0.9867 - val_accuracy: 0.9458 - val_loss: 0.1616


In [12]:
model.export("C:/Users/parth/Desktop/Desktop/Concordia/6180/Saved_model")

INFO:tensorflow:Assets written to: C:/Users/parth/Desktop/Desktop/Concordia/6180/Saved_model\assets


INFO:tensorflow:Assets written to: C:/Users/parth/Desktop/Desktop/Concordia/6180/Saved_model\assets


Saved artifact at 'C:/Users/parth/Desktop/Desktop/Concordia/6180/Saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name='keras_tensor_1233')
Output Type:
  TensorSpec(shape=(None, 25), dtype=tf.float32, name=None)
Captures:
  2097080556832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080554368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080555248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080556656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080554896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080559296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080554720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2097080554016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2101552832480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2101552833184: TensorSpec(shape=(), dtype=tf.resource, 